In [53]:
# basic set ups, functions

from functools import partial
import xarray as xr
import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd
import numpy as np

import cartopy.crs as ccrs
import cartopy.feature as cf

import os
import rioxarray
import dask

def load_filenames(fp, scenarios):
    fns = []
    for scenario in scenarios:
            fn = f"%s%s/total_%s_medium_confidence_values.nc"%(fp,'GMSL', scenario)
            fns.append(fn)
    return fns

def _preprocess_gmsl(ds):
    ds['scenario'] = '0'
    return ds

def read_cmip6_tas_scenario():
    """read scenario-based tas projection (from CMIP6) into one dataframe

    Returns:
        dataframe: a dataframe containing three columns: Year, Mean, Scenario
    """
    fp = "./cmip6_tas_scenario"
    df = pd.DataFrame()
    scenarios = ('historical','ssp119','ssp126', 'ssp245', 'ssp370', 'ssp585')
    for scenario in scenarios:
        fn = os.path(["%s/tas_global_%s.csv"%(fp,str.upper(scenario))])
        df_this = pd.read_csv(fn,dtype={'Year':np.int32})[['Year','Mean']]
        df_this['scenario'] = scenario
        df = pd.concat([df,df_this], ignore_index=True)
    return df

def load_raw_data(fp,scenarios,loc_str):

    fns = load_filenames(fp, scenarios)
    partial_func = partial(lambda x: _preprocess_gmsl(x))   
    #reading the zarr file
    ds = xr.open_mfdataset(fns, parallel=True, preprocess=partial_func, combine = 'nested', concat_dim = 'scenario')

    ds.scenario
    ds = ds.assign_coords({"scenario": scenarios})
    return ds

fp = "./"
scenarios = ['ssp119','ssp126','ssp245','ssp370','ssp585']

loc_str = 'global_grids'
ds = load_raw_data(fp, scenarios, loc_str)
ds = ds.drop(["lat","lon"]).mean('locations')
ds_interp = ds.interp(years = np.arange(2010,2101), method="linear",kwargs={'fill_value':'extrapolate'}).compute()
ds_interp = ds_interp.chunk(chunks = {"scenario":5, "quantiles":"auto", "years":"auto"})

ds_interp


<xarray.Dataset>
Dimensions:           (scenario: 5, quantiles: 107, years: 91)
Coordinates:
  * quantiles         (quantiles) float32 0.0 0.001 0.005 ... 0.995 0.999 1.0
  * scenario          (scenario) <U6 'ssp119' 'ssp126' ... 'ssp370' 'ssp585'
  * years             (years) int64 2010 2011 2012 2013 ... 2097 2098 2099 2100
Data variables:
    sea_level_change  (scenario, quantiles, years) float64 dask.array<chunksize=(5, 107, 91), meta=np.ndarray>

In [59]:
ds_interp.to_zarr("s3://prod-climate-scratch/SLR/gmsl_2010_2100.zarr",consolidated=True)

In [61]:
ds_interp = xr.open_zarr("s3://prod-climate-scratch/SLR/gmsl_2010_2100.zarr")